In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 8.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="7HarG4EIF3gVxdmGryIh")
project = rf.workspace("leaf1").project("rhizome-rot-detection")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov9c.pt")  # load a pretrained model (recommended for training)


100%|██████████| 49.4M/49.4M [00:00<00:00, 356MB/s]


In [ ]:
results = model.train(data="/content/RHizome-rot-detection-1/data.yaml", epochs=50, imgsz=640, batch=8)

In [ ]:
from ultralytics import YOLO
model = YOLO('/content/runs/detect/train/weights/best.pt')

results = model.predict("/content/RHizome-rot-detection-1/valid/images/10R_jpg.rf.bf0bb30a3b1d0fe095c8cd4ced0db068.jpg",imgsz=640,conf=0.3,save=True,show=True)

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Load your trained model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Function to get predictions
def get_predictions(image_path):
    results = model.predict(image_path, imgsz=640, conf=0.3, save=True, show=False)
    predictions = []
    for result in results:
        for box in result.boxes:
            # Extracting the coordinates correctly
            x_center, y_center, width, height = box.xywh[0].tolist()
            predictions.append({
                'class': box.cls.item(),
                'x': x_center,
                'y': y_center,
                'w': width,
                'h': height,
                'confidence': box.conf.item()
            })
    return predictions

# Function to determine plant health
def determine_health(predictions):
    for pred in predictions:
        if pred['class'] == 0:  # Assuming class 0 is 'infected'
            return True
    return False

# Function to process and display the image
def process_image(image_path, predictions):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')

    if determine_health(predictions):
        plt.title("The plant is affected by rhizome rot")
        for pred in predictions:
            if pred['class'] == 0:  # Only show infected boxes
                rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor='r', facecolor='none')
                plt.gca().add_patch(rect)
    else:
        plt.title("The plant is healthy")
        for pred in predictions:
            color = 'r' if pred['class'] == 0 else 'g'  # Red for infected, green for healthy
            rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor=color, facecolor='none')
            plt.gca().add_patch(rect)

    plt.show()

# Example usage
image_path = "/content/8R.jpg"
predictions = get_predictions(image_path)
process_image(image_path, predictions)


In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Load your trained model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Function to get predictions
def get_predictions(image_path):
    results = model.predict(image_path, imgsz=640, conf=0.3, save=True, show=False)
    predictions = []
    for result in results:
        for box in result.boxes:
            # Extracting the coordinates correctly
            x_center, y_center, width, height = box.xywh[0].tolist()
            predictions.append({
                'class': box.cls.item(),
                'x': x_center,
                'y': y_center,
                'w': width,
                'h': height,
                'confidence': box.conf.item()
            })
    return predictions

# Function to determine plant health and infection stage
def determine_health_and_stage(predictions):
    infected_count = sum(1 for pred in predictions if pred['class'] == 0)  # Assuming class 0 is 'infected'
    if infected_count == 0:
        return "The plant is healthy"
    elif 1 <= infected_count <= 2:
        return "The plant is infected by rhizome rot - Stage 1: Initial Infection"
    elif infected_count == 3:
        return "The plant is infected by rhizome rot - Stage 2: Moderate Disease Progression"
    else:
        return "The plant is infected by rhizome rot - Stage 3: Severe Infection"

# Function to process and display the image
def process_image(image_path, predictions):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')

    stage = determine_health_and_stage(predictions)
    plt.title(stage)

    for pred in predictions:
        if pred['class'] == 0:  # Only show infected boxes
            rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor='r', facecolor='none')
            plt.gca().add_patch(rect)
        else:
            rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor='g', facecolor='none')
            plt.gca().add_patch(rect)

    plt.show()

# Example usage
image_path = "/content/8R.jpg"
predictions = get_predictions(image_path)
process_image(image_path, predictions)


In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Load your trained model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Function to get predictions
def get_predictions(image_path):
    results = model.predict(image_path, imgsz=640, conf=0.3, save=True, show=False)
    predictions = []
    for result in results:
        for box in result.boxes:
            # Extracting the coordinates correctly
            x_center, y_center, width, height = box.xywh[0].tolist()
            predictions.append({
                'class': box.cls.item(),
                'x': x_center,
                'y': y_center,
                'w': width,
                'h': height,
                'confidence': box.conf.item()
            })
    return predictions

# Function to determine plant health and infection stage
def determine_health_and_stage(predictions):
    infected_count = sum(1 for pred in predictions if pred['class'] == 0)  # Assuming class 0 is 'infected'
    if infected_count == 0:
        return "The plant is healthy"
    elif 1 <= infected_count <= 2:
        return "The plant is infected by rhizome rot - Stage 1: Initial Infection"
    elif 3 <= infected_count <= 4:
        return "The plant is infected by rhizome rot - Stage 2: Moderate Disease Progression"
    else:
        return "The plant is infected by rhizome rot - Stage 3: Severe Infection"

# Function to process and display the image
def process_image(image_path, predictions):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')

    stage = determine_health_and_stage(predictions)
    plt.title(stage)

    for pred in predictions:
        if pred['class'] == 0:  # Only show infected boxes
            rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor='r', facecolor='none')
            plt.gca().add_patch(rect)
        else:
            rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor='g', facecolor='none')
            plt.gca().add_patch(rect)

    plt.show()

# Example usage
image_path = "/content/2R.jpg"
predictions = get_predictions(image_path)
process_image(image_path, predictions)


In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Load your trained model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Function to get predictions
def get_predictions(image_path):
    results = model.predict(image_path, imgsz=640, conf=0.3, save=True, show=False)
    predictions = []
    for result in results:
        for box in result.boxes:
            # Extracting the coordinates correctly
            x_center, y_center, width, height = box.xywh[0].tolist()
            predictions.append({
                'class': box.cls.item(),
                'x': x_center,
                'y': y_center,
                'w': width,
                'h': height,
                'confidence': box.conf.item()
            })
    return predictions

# Function to determine plant health and infection stage
def determine_health_and_stage(predictions):
    infected_count = sum(1 for pred in predictions if pred['class'] == 0)  # Assuming class 0 is 'infected'
    if infected_count == 0:
        return "The plant is healthy"
    elif 1 <= infected_count <= 2:
        return "The plant is infected by rhizome rot - Stage 1: Initial Infection"
    elif 3 <= infected_count <= 4:
        return "The plant is infected by rhizome rot - Stage 2: Moderate Disease Progression"
    else:
        return "The plant is infected by rhizome rot - Stage 3: Severe Infection"

# Function to process and display the image
def process_image(image_path, predictions):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')

    stage = determine_health_and_stage(predictions)
    plt.title(stage)

    for pred in predictions:
        if pred['class'] == 0:  # Only show infected boxes
            rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor='r', facecolor='none')
            plt.gca().add_patch(rect)
        else:
            rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor='g', facecolor='none')
            plt.gca().add_patch(rect)

    plt.show()

# Example usage
image_path = "/content/10R.jpg"
predictions = get_predictions(image_path)
process_image(image_path, predictions)


In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Load your trained model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Function to get predictions
def get_predictions(image_path):
    results = model.predict(image_path, imgsz=640, conf=0.3, save=True, show=False)
    predictions = []
    for result in results:
        for box in result.boxes:
            # Extracting the coordinates correctly
            x_center, y_center, width, height = box.xywh[0].tolist()
            predictions.append({
                'class': box.cls.item(),
                'x': x_center,
                'y': y_center,
                'w': width,
                'h': height,
                'confidence': box.conf.item()
            })
    return predictions

# Function to determine plant health and infection stage
def determine_health_and_stage(predictions):
    infected_count = sum(1 for pred in predictions if pred['class'] == 0)  # Assuming class 0 is 'infected'
    if infected_count == 0:
        return "The plant is healthy"
    elif 1 <= infected_count <= 2:
        return "The plant is infected by rhizome rot - Stage 1: Initial Infection"
    elif 3 <= infected_count <= 4:
        return "The plant is infected by rhizome rot - Stage 2: Moderate Disease Progression"
    else:
        return "The plant is infected by rhizome rot - Stage 3: Severe Infection"

# Function to process and display the image
def process_image(image_path, predictions):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')

    stage = determine_health_and_stage(predictions)
    plt.title(stage)

    for pred in predictions:
        if pred['class'] == 0:  # Only show infected boxes
            rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor='r', facecolor='none')
            plt.gca().add_patch(rect)
        else:
            rect = patches.Rectangle((pred['x'] - pred['w']/2, pred['y'] - pred['h']/2), pred['w'], pred['h'], linewidth=2, edgecolor='g', facecolor='none')
            plt.gca().add_patch(rect)

    plt.show()

# Example usage
image_path = "/content/11.jpg"
predictions = get_predictions(image_path)
process_image(image_path, predictions)
